# NL to FOL Translation with `LiteLLM`

Use `LiteLLM` to translate natural language sentences to first-order logic (FOL) formulas, which offers more flexibility to swap the underlying LLM.

## 0. Setup

In [29]:
import os
import json
import pandas as pd
from litellm import completion
from rich import print as rprint

from prompt import PromptTemplate, TRANSLATE_TEMPLATE_V1, TRANSLATE_EXAMPLES_V1, CORRECT_TEMPLATE_V1
from metrics import is_syntactically_valid_with_timeout

In [3]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
trans_prompt_template = PromptTemplate(TRANSLATE_TEMPLATE_V1)
correct_prompt_template = PromptTemplate(CORRECT_TEMPLATE_V1)

In [8]:
df = pd.read_csv('data/quy_che.tsv', sep='\t')

all_quy_che = df['quy_che'].tolist()
all_quy_che_fols = []

In [48]:
for nl_str in all_quy_che:
    messages = []
    
    trans_prompt = trans_prompt_template(
        nl_str=nl_str,
        existing_predicates_str=[],
        examples_str=TRANSLATE_EXAMPLES_V1
    )
    messages.append({
        "role": "user",
        "content": trans_prompt
    })
    rprint(messages[-1]) # FIXME: delete this line
    invalid_premises = []
    
    while True:
        response = completion(
            model=os.environ['LITELLM_MODEL'],
            messages=messages,
            api_key=os.environ['LITELLM_API_KEY'], 
            base_url=os.environ['LITELLM_BASE_URL'],
            # temperature=0.0,
            # top_p=0.7,
            # max_tokens=8192,
            stream=False
        )

        result = response.choices[0].message.content
        messages.append({
            "role": "assistant",
            "content": result
        })
        rprint(messages[-1]) # FIXME: delete this line
        
        json_str = result.strip('```').strip()
        if json_str.startswith('json'):
            json_str = json_str[4:].strip()
        parsed_json = json.loads(json_str)
        
        invalid_premises = [] # Reset invalid_premises
        for premise in parsed_json['premises']:
            if not is_syntactically_valid_with_timeout(premise):
                invalid_premises.append(premise)
        
        if len(invalid_premises) > 0:
            messages.append({
                "role": "user",
                "content": correct_prompt_template(invalid_premises_str='- ' + '\n- '.join(invalid_premises))
            })
            rprint(messages[-1]) # FIXME: delete this line
        else:
            all_quy_che_fols.append(parsed_json)
            break
    

{
    'role': 'user',
    'content': 'Translate the following natural language (NL) statement to a first-order logic (FOL) rule.\n\nWhen 
there are pre-defined predicates, try to use them to form the premises, and only add new predicates if 
necessary.\n\nThe output must be in JSON format and has the following 3 fields:\n* `existing_predicates`: array of 
existing predicates that can be used to form the premises, in camel case with no space, and number of variables it 
takes e.g., `CamelCase(x,y)`\n* `new_predicates`: array of new predicates, in camel case with no space, and number 
of variables it takes e.g., `CamelCase(x,y)`\n* `variables`: array of variables, in lower case with no space, e.g.,
`lowercase`\n* `premises`: array of premises constructed from the NL statement\n* `conclusion`: the translation of 
the conclusion of the NL, can be a question or a statement. If it is a yes/no question, translate to the one with 
\'yes\' semantic meaning.\n\nImportant notes:\n* Don\'t forget to also include predicates used in `conclusion` in 
either `existing_predicates` or `new_predicates`\n* In FOL logic, there are no operators like <, >, =, etc. Define 
predicates for them instead.\n\nPlease return only the JSON output, don\'t include any other text.\n\n--- Start of 
Example 1 ---\nNL:\nAll people who regularly drink coffee are dependent on caffeine. People regularly drink coffee,
or they don\'t want to be addicted to caffeine, or both. No one who doesn\'t want to be addicted to caffeine is 
unaware that caffeine is a drug. Rina is either a student who is unaware that caffeine is a drug, or she is not a 
student and is she aware that caffeine is a drug. Rina is either a student who is dependent on caffeine, or she is 
not a student and not dependent on caffeine.\n\nExisting predicates:\nNone\n\nFOL translation:\n```\n{\n  
"existing_predicates": [],\n  "new_predicates": [\n    "DrinkRegularly(x,y)",\n    "IsDependentOn(x,y)",\n    
"DrinkRegularly(x,y)",\n    "WantToBeAddictedTo(x,y)",\n    "AwareThatDrug(x,y)",\n    "Student(x)"\n  ],\n  
"variables": [\n    "coffee",\n    "caffeine",\n    "rina"\n  ],\n  "premises": [\n    "∀x (DrinkRegularly(x, 
coffee) → IsDependentOn(x, caffeine))",\n    "∀x (DrinkRegularly(x, coffee) ∨ (¬WantToBeAddictedTo(x, 
caffeine)))",\n    "∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕ 
¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina))"\n  ],\n  "conclusion": 
""\n}\n```\n--- End of Example 1 ---\n\n--- Start of Example 2 ---\nNL:\nAll people who regularly drink coffee are 
dependent on caffeine. People regularly drink coffee, or they don\'t want to be addicted to caffeine, or both. No 
one who doesn\'t want to be addicted to caffeine is unaware that caffeine is a drug. Rina is either a student who 
is unaware that caffeine is a drug, or she is not a student and is she aware that caffeine is a drug. Rina is 
either a student who is dependent on caffeine, or she is not a student and not dependent on caffeine.\n\nExisting 
predicates:\n- DrinkRegularly(x,y)\n- IsDependentOn(x,y)\n- WantToBeAddictedTo(x,y)\n\nFOL translation:\n```\n{\n  
"existing_predicates": [\n    "DrinkRegularly(x,y)",\n    "IsDependentOn(x,y)",\n    "WantToBeAddictedTo(x,y)"\n  
],\n  "new_predicates": [\n    "AwareThatDrug(x,y)",\n    "Student(x)"\n  ],\n  "variables": [\n    "coffee",\n    
"caffeine",\n    "rina"\n  ],\n  "premises": [\n    "∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, 
caffeine))",\n    "∀x (DrinkRegularly(x, coffee) ∨ (¬WantToBeAddictedTo(x, caffeine)))",\n    "∀x 
(¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕ ¬AwareThatDrug(rina, caffeine)) 
¬(IsDependentOn(rina, caffeine) ⊕ Student(rina))"\n  ],\n  "conclusion": ""\n}\n```\n--- End of Example 2 
---\n\n\nNL:\nCác học phần tốt nghiệp (Thực tập ngoài trường, Đồ án chuyên ngành, Khoá luận tốt nghiệp) được thiết
kế vào cuối chươ

{
    'role': 'assistant',
    'content': '```json\n{\n  "existing_predicates": [],\n  "new_predicates": [\n    "GraduationCourse(x)",\n    
"DesignedAtEnd(x)",\n    "CompletedLanguageRequirement(x)",\n    "CompletedCreditRequirement(x)",\n    
"CanRegister(x,y)",\n    "AccumulatedCredits(x,y)",\n    "TotalCredits(x)",\n    "CompletedTrainingProgram(x,y)",\n
"MetGraduationLanguageStandard(x)",\n    "ConcurrentCourse(x,y)",\n    "PrerequisiteCourse(x,y)"\n  ],\n  
"variables": [\n    "x",\n    "y",\n    "internship",\n    "specializedProject",\n    "graduationThesis",\n    
"student"\n  ],\n  "premises": [\n    "∀x (GraduationCourse(x) → DesignedAtEnd(x))",\n    "∀x ∀y 
((CompletedLanguageRequirement(x) ∧ CompletedCreditRequirement(x)) → CanRegister(x,y))",\n    "∀x 
(AccumulatedCredits(x, 0.65 * TotalCredits(x)) → CanRegister(x, internship))",\n    "∀x (AccumulatedCredits(x, 0.70
* TotalCredits(x)) → CanRegister(x, specializedProject))",\n    "∀x ((AccumulatedCredits(x, 0.85 * TotalCredits(x))
∧ CompletedTrainingProgram(x, 0.70) ∧ MetGraduationLanguageStandard(x)) → CanRegister(x, graduationThesis))",\n    
"ConcurrentCourse(internship, specializedProject)",\n    "PrerequisiteCourse(internship, graduationThesis)",\n    
"PrerequisiteCourse(specializedProject, graduationThesis)"\n  ],\n  "conclusion": ""\n}\n```'
}

{
    'role': 'user',
    'content': 'There are some invalid premises:\n\n- ∀x (AccumulatedCredits(x, 0.65 * TotalCredits(x)) → 
CanRegister(x, internship))\n- ∀x (AccumulatedCredits(x, 0.70 * TotalCredits(x)) → CanRegister(x, 
specializedProject))\n- ∀x ((AccumulatedCredits(x, 0.85 * TotalCredits(x)) ∧ CompletedTrainingProgram(x, 0.70) ∧ 
MetGraduationLanguageStandard(x)) → CanRegister(x, graduationThesis))\n\nPlease check, correct them and provide a 
new translation in the correct format (no further explanation needed). You can redefine the predicates if needed. 
Some tips:\n* In FOL logic, there are no mathematic operators like <, >, =, etc. Define predicates for them 
instead. For example, `Joe has age less than 30 years old` can be translated as `LessThan30YearsOld(joe)`, etc.\n* 
Always check for number of parentheses and ensure each open parenthesis should have a corresponding close 
parenthesis.\n* Nested predicates e.g., `P1(P2(x))` are invalid. Instead, you should define new variable and/or 
predicate to represent the natural language statement.\n* Please use `→` only, and do NOT use the backward version 
`←` for implication.'
}

{
    'role': 'assistant',
    'content': '```json\n{\n  "existing_predicates": [],\n  "new_predicates": [\n    "GraduationCourse(x)",\n    
"DesignedAtEnd(x)",\n    "CompletedLanguageRequirement(x)",\n    "CompletedCreditRequirement(x)",\n    
"CanRegister(x,y)",\n    "AccumulatedCreditsAtLeast65Percent(x)",\n    "AccumulatedCreditsAtLeast70Percent(x)",\n  
"AccumulatedCreditsAtLeast85Percent(x)",\n    "CompletedTrainingProgramAtLeast70Percent(x)",\n    
"MetGraduationLanguageStandard(x)",\n    "ConcurrentCourse(x,y)",\n    "PrerequisiteCourse(x,y)"\n  ],\n  
"variables": [\n    "x",\n    "y",\n    "internship",\n    "specializedProject",\n    "graduationThesis"\n  ],\n  
"premises": [\n    "∀x (GraduationCourse(x) → DesignedAtEnd(x))",\n    "∀x ∀y ((CompletedLanguageRequirement(x) ∧ 
CompletedCreditRequirement(x)) → CanRegister(x,y))",\n    "∀x (AccumulatedCreditsAtLeast65Percent(x) → 
CanRegister(x, internship))",\n    "∀x (AccumulatedCreditsAtLeast70Percent(x) → CanRegister(x, 
specializedProject))",\n    "∀x ((AccumulatedCreditsAtLeast85Percent(x) ∧ 
CompletedTrainingProgramAtLeast70Percent(x) ∧ MetGraduationLanguageStandard(x)) → CanRegister(x, 
graduationThesis))",\n    "ConcurrentCourse(internship, specializedProject)",\n    "PrerequisiteCourse(internship, 
graduationThesis)",\n    "PrerequisiteCourse(specializedProject, graduationThesis)"\n  ],\n  "conclusion": 
""\n}\n```'
}

In [ ]:
for i, quy_che_fol in enumerate(all_quy_che_fols):
    rprint(f"Quy chế {i+1}:")
    rprint(quy_che_fol)